# Scikit-learn Comparison

fuente: https://sklearn-genetic-opt.readthedocs.io/en/stable/index.html


In [ ]:
from sklearn_genetic import GASearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn_genetic.space import Categorical, Continuous
import scipy.stats as stats
from sklearn.utils.fixes import loguniform
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = load_digits() 

In [ ]:
# Display the last digit
plt.figure(1, figsize=(3, 3))
plt.imshow(data.images[-1], cmap=plt.cm.gray_r, interpolation="nearest")
plt.show()

In [ ]:
label_names = data['target_names'] 
y = data['target']
X = data['data'] 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
clf = SGDClassifier(loss='hinge',fit_intercept=True)

### 1. Random Search

In [ ]:
param_dist = {'average': [True, False],
              'l1_ratio': stats.uniform(0, 1),
              'alpha': loguniform(1e-4, 1e0)}

In [ ]:
print(stats.uniform(0, 1).rvs(size=5))
print(loguniform.rvs(1e-4, 1e0, size=4))

In [ ]:
n_iter_search = 30
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search,n_jobs=-1)

In [ ]:
%%time
random_search.fit(X_train,y_train)

In [ ]:
accuracy_score(y_test, random_search.predict(X_test))

In [ ]:
random_search.best_params_

### 2. Grid Search

In [ ]:
param_grid = {'average': [True, False],
              'l1_ratio': np.linspace(0, 1, num=10),
              'alpha': np.power(10, np.arange(-4, 1, dtype=float))}

In [ ]:
grid_search = GridSearchCV(clf, param_grid=param_grid,n_jobs=-1)

In [ ]:
%%time
grid_search.fit(X_train,y_train)

In [ ]:
accuracy_score(y_test, grid_search.predict(X_test))

In [ ]:
grid_search.best_params_

### 3. Genetic Algorithm

In [ ]:
param_grid = {'l1_ratio': Continuous(0,1),
              'alpha': Continuous(1e-4,1),
              'average': Categorical([True, False])}

evolved_estimator = GASearchCV(clf,
                    cv=3,
                    scoring='accuracy',
                    param_grid=param_grid,
                    population_size=10,
                    generations=8,
                    tournament_size=3,
                    elitism=True,
                    verbose=True)

In [ ]:
%%time
evolved_estimator.fit(X_train,y_train)

In [ ]:
y_predicy_ga = evolved_estimator.predict(X_test)

In [ ]:
accuracy_score(y_test,y_predicy_ga)

In [ ]:
evolved_estimator.best_params_